In [3]:
# Imports and configuration
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pennylane as qml
from pennylane import numpy as pnp
import time
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

class QuantumEncodingComparison:
    def __init__(self, n_qubits=8, n_layers=3, learning_rate=0.01, epochs=50):
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.dev = qml.device("default.qubit", wires=n_qubits)
        self.results = {}

    def prepare_data(self):
        data = load_breast_cancer()
        X = data.data
        y = data.target
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
        X_train = pnp.array(X_train[:, :self.n_qubits], requires_grad=False)
        X_test = pnp.array(X_test[:, :self.n_qubits], requires_grad=False)
        y_train = pnp.array(y_train, requires_grad=False)
        y_test = pnp.array(y_test, requires_grad=False)
        return X_train, X_test, y_train, y_test

    def angle_encoding_circuit(self, inputs, weights):
        qml.AngleEmbedding(inputs, wires=range(self.n_qubits), rotation='Y')
        qml.StronglyEntanglingLayers(weights, wires=range(self.n_qubits))
        return qml.expval(qml.PauliZ(0))

    def amplitude_encoding_circuit(self, inputs, weights):
        inputs_np = pnp.array(inputs)
        norm = pnp.linalg.norm(inputs_np)
        normalized_inputs = inputs_np / (norm + 1e-8)
        state_size = 2 ** self.n_qubits

        if len(normalized_inputs) > state_size:
            print(f"Warning: Truncating input features from {len(normalized_inputs)} to {state_size} for amplitude encoding.")
            padded_inputs = normalized_inputs[:state_size]
        else:
            padded_inputs = pnp.zeros(state_size)
            padded_inputs[:len(normalized_inputs)] = normalized_inputs

        norm_final = pnp.linalg.norm(padded_inputs)
        padded_inputs = padded_inputs / (norm_final + 1e-8)

        qml.MottonenStatePreparation(padded_inputs, wires=range(self.n_qubits))
        qml.StronglyEntanglingLayers(weights, wires=range(self.n_qubits))
        return qml.expval(qml.PauliZ(0))

    def hybrid_encoding_circuit(self, inputs, weights):
        half_features = len(inputs) // 2
        angle_features = inputs[:half_features]
        for i, feature in enumerate(angle_features):
            if i < self.n_qubits:
                qml.RY(feature, wires=i)
        remaining_features = inputs[half_features:]
        for i, feature in enumerate(remaining_features):
            if i + half_features < self.n_qubits:
                qml.RZ(feature, wires=i + half_features)
        qml.StronglyEntanglingLayers(weights, wires=range(self.n_qubits))
        return qml.expval(qml.PauliZ(0))

    def dense_encoding_circuit(self, inputs, weights):
        features_per_qubit = len(inputs) // self.n_qubits + (1 if len(inputs) % self.n_qubits != 0 else 0)
        for qubit in range(self.n_qubits):
            start_idx = qubit * features_per_qubit
            end_idx = min(start_idx + features_per_qubit, len(inputs))
            qubit_features = inputs[start_idx:end_idx]
            for j, feature in enumerate(qubit_features):
                if j == 0:
                    qml.RY(feature, wires=qubit)
                elif j == 1:
                    qml.RZ(feature, wires=qubit)
                elif j == 2:
                    qml.RX(feature, wires=qubit)
        qml.StronglyEntanglingLayers(weights, wires=range(self.n_qubits))
        return qml.expval(qml.PauliZ(0))

    def create_qnode(self, encoding_type):
        if encoding_type == 'angle':
            return qml.QNode(self.angle_encoding_circuit, self.dev)
        elif encoding_type == 'amplitude':
            return qml.QNode(self.amplitude_encoding_circuit, self.dev)
        elif encoding_type == 'hybrid':
            return qml.QNode(self.hybrid_encoding_circuit, self.dev)
        elif encoding_type == 'dense':
            return qml.QNode(self.dense_encoding_circuit, self.dev)
        else:
            raise ValueError(f"Unknown encoding type: {encoding_type}")

    def train_model(self, circuit_fn, X_train, y_train):
        weights = pnp.array(0.01 * np.random.randn(self.n_layers, self.n_qubits, 3), requires_grad=True)

        def cost(weights):
            predictions = [circuit_fn(x, weights) for x in X_train]
            return pnp.mean((predictions - y_train) ** 2)

        opt = qml.GradientDescentOptimizer(stepsize=self.learning_rate)
        loss_history = []
        acc_history = []

        for epoch in range(self.epochs):
            weights = opt.step(cost, weights)
            predictions = [circuit_fn(x, weights) for x in X_train]
            pred_labels = [1 if p >= 0.5 else 0 for p in predictions]
            acc = accuracy_score(y_train, pred_labels)
            loss = cost(weights)
            loss_history.append(loss)
            acc_history.append(acc)
            if epoch % 10 == 0 or epoch == self.epochs - 1:
                print(f"Epoch {epoch+1}: Loss = {loss:.4f}, Accuracy = {acc:.4f}")

        return weights, loss_history, acc_history

    def evaluate_model(self, circuit_fn, weights, X_test, y_test):
        predictions = [circuit_fn(x, weights) for x in X_test]
        pred_labels = [1 if p >= 0.5 else 0 for p in predictions]
        accuracy = accuracy_score(y_test, pred_labels)
        precision = precision_score(y_test, pred_labels)
        recall = recall_score(y_test, pred_labels)
        f1 = f1_score(y_test, pred_labels)
        return accuracy, precision, recall, f1

    def run_comparison(self):
        X_train, X_test, y_train, y_test = self.prepare_data()
        encoding_types = ['angle', 'amplitude', 'hybrid', 'dense']

        for enc in encoding_types:
            print("\n" + "=" * 50)
            print(f"Processing {enc.upper()} encoding")
            print("=" * 50)
            qnode = self.create_qnode(enc)
            circuit = lambda x, w: qnode(x, w)
            start = time.time()
            weights, loss_hist, acc_hist = self.train_model(circuit, X_train, y_train)
            end = time.time()
            acc, prec, rec, f1 = self.evaluate_model(circuit, weights, X_test, y_test)

            self.results[enc] = {
                'test_metrics': {
                    'accuracy': acc,
                    'precision': prec,
                    'recall': rec,
                    'f1_score': f1
                },
                'training_time': end - start,
                'loss_history': loss_hist,
                'acc_history': acc_hist
            }
            print(f"Training completed in {end - start:.2f} seconds")
            print(f"Final test accuracy: {acc:.4f}")

    def plot_results(self):
        for metric in ['loss_history', 'acc_history']:
            plt.figure(figsize=(10, 6))
            for enc, result in self.results.items():
                plt.plot(result[metric], label=enc)
            plt.title(metric.replace('_', ' ').title())
            plt.xlabel("Epoch")
            plt.ylabel(metric.split('_')[0].capitalize())
            plt.legend()
            plt.grid(True)
            plt.show()

    def generate_report(self):
        rows = []
        for enc, result in self.results.items():
            metrics = result['test_metrics']
            row = {
                'Encoding': enc,
                'Accuracy': metrics['accuracy'],
                'Precision': metrics['precision'],
                'Recall': metrics['recall'],
                'F1 Score': metrics['f1_score'],
                'Training Time (s)': result['training_time']
            }
            rows.append(row)
        return pd.DataFrame(rows)
